### European Soccer

#### Objective: Build a predictive model for soccer outcomes (i.e. win/loss/tie) that out-performs the base case and approaches or surpasses the experts.


#### Potential predictors to use:

1.) Home Team Historic Win % (Last 1 Wk, Last 2 Wks, Last yr, etc...)

2.) Away Team Historic Win % 

3.) Home Total Player Rating

4.) Away Total Player Rating

5.) Historic Matchup Outcome (i.e. between specific teams)

6.) Home Field Advantage

7.) Bookie Predictions (?)

    -> Potentially grade each bookie and weight their predictions based on historical success similar to Turk HW

#### Compare Model Results Against:

1.) Actual Outcomes

2.) Aggregate Bookie Odds

#### Eliminate from data:

1.) Formations

2.) Detailed in game statistics (i.e. corners, shots on goal, fouls, etc...) (?)

2.) All player ratings but the Overall Rating (?)

In [54]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
import os

In [60]:
with sqlite3.connect('soccer\database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    player = pd.read_sql_query("SELECT * from Player", con)
    playerstats = pd.read_sql_query("SELECT player_api_id, date_stat, overall_rating \
                                    FROM Player_Stats \
                                    WHERE overall_rating is not NULL", con)

In [61]:
#Pull in countries, leagues & teams from other data tables
countries = countries.set_index('id')
leagues = leagues.set_index('id')
teams = teams.set_index('team_api_id')

matches['country'] = matches['country_id'].map(countries['name'])
matches['league'] = matches['league_id'].map(leagues['name'])
matches['home_team'] = matches['home_team_api_id'].map(teams['team_long_name'])
matches['away_team'] = matches['away_team_api_id'].map(teams['team_long_name'])

In [62]:
def get_winner(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 'home'
    elif row['home_team_goal'] < row['away_team_goal']:
        return 'away'
    else:
        return 'tie'

matches['winner'] = matches.apply(lambda row: get_winner(row), axis = 1)
matches.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,country,league,home_team,away_team,winner
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,3.25,4.00,1.73,3.40,4.20,Belgium,Belgium Jupiler League,Genk,Beerschot AC,tie
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,3.75,1.91,3.25,3.60,Belgium,Belgium Jupiler League,Zulte-Waregem,Lokeren,tie
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,NaN,NaN,NaN,NaN,NaN,Belgium,Belgium Jupiler League,Cercle Bruges,Anderlecht,away
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,3.75,5.50,1.44,3.75,6.50,Belgium,Belgium Jupiler League,Gent,Mons,home
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,3.50,1.65,4.75,3.30,1.67,Belgium,Belgium Jupiler League,FCV Dender EH,Standard Liege,away


In [63]:
#Drop Key IDs, player position columns and in-game statistics (i.e. shots, fouls, crosses, etc...) as out of scope of analysis
matches = matches.drop(matches.columns[77:85], axis = 1)
matches = matches.drop(matches.columns[11:55], axis = 1)
matches = matches.drop(matches.columns[[0,1,2,6,7,8]], axis = 1)

matches.columns.values

array(['season', 'stage', 'date', 'home_team_goal', 'away_team_goal',
       'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH',
       'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
       'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH',
       'BSD', 'BSA', 'country', 'league', 'home_team', 'away_team',
       'winner'], dtype=object)

In [64]:
"""
Detailed attributes from the EA Sports FIFA video game series. Players stats are updated frequently and paint 
a fairly accurate picture of the player's skillset. Unlike Baseball, soccer doesn't have a lot of consistent statistics to measure 
actual performance on the field, but this is a decent proxy (though obviously subject to the biases of the EA Sports team 
assessing player's ratings)
"""
playerstats.head(5)

,player_api_id,date_stat,overall_rating
0,505942,2016-02-18 00:00:00,67
1,505942,2015-11-19 00:00:00,67
2,505942,2015-09-21 00:00:00,62
3,505942,2015-03-20 00:00:00,61
4,505942,2007-02-22 00:00:00,61


In [65]:
matches.head(5)

,season,stage,date,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,...,GBD,GBA,BSH,BSD,BSA,country,league,home_team,away_team,winner
0,2008/2009,1,2008-08-17 00:00:00,1,1,NaN,NaN,NaN,NaN,NaN,...,3.25,4.00,1.73,3.40,4.20,Belgium,Belgium Jupiler League,Genk,Beerschot AC,tie
1,2008/2009,1,2008-08-16 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,...,3.25,3.75,1.91,3.25,3.60,Belgium,Belgium Jupiler League,Zulte-Waregem,Lokeren,tie
2,2008/2009,1,2008-08-16 00:00:00,0,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Belgium,Belgium Jupiler League,Cercle Bruges,Anderlecht,away
3,2008/2009,1,2008-08-17 00:00:00,5,0,NaN,NaN,NaN,NaN,NaN,...,3.75,5.50,1.44,3.75,6.50,Belgium,Belgium Jupiler League,Gent,Mons,home
4,2008/2009,1,2008-08-16 00:00:00,1,3,NaN,NaN,NaN,NaN,NaN,...,3.50,1.65,4.75,3.30,1.67,Belgium,Belgium Jupiler League,FCV Dender EH,Standard Liege,away


In [66]:
#list of unique players and attributes
player.head(5)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
